In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))

# 0 - Read prepped cohorts data

- Read in prepared data and output

In [ ]:
results <- fread(paste0(SHARE_DIR, "2b_go.csv")) 
fisher_base <- fread(paste0(SHARE_DIR, "fisher_base.csv"))

- Collect top independent features

In [ ]:
results <- 
results %>% mu( p_bh = p.adjust(fisher_pval, method = "BH"), p_by = p.adjust(fisher_pval, method = "BY")) %>%
 gb(cohortGo) %>% mu(max_patients = max(total_patients))

In [ ]:
tots <- 
results %>% 
 gb(cohortGo, max_patients, type) %>% 
 su( p_sig = sum(fisher_pval < .05), 
     p_bh_sig = sum(p_bh < .05),
     p_by_sig = sum(p_by < .05), 
     tests = n(), 
     type1 = p_sig/tests, 
     lt_05 = type1 < .05) %>% 
 ug() %>% 
 ar(p_bh_sig) 

- Highlighters

In [ ]:
highlighters_univariate <-
c(
 "Pan-Cancer ## Pazopanib" = "rna_geneset_KEGG_PHENYLALANINE_METABOLISM_lt25", 
 "Prostate ## Abiraterone" = "rna_geneset_KEGG_TGF_BETA_SIGNALING_PATHWAY_gt25", 
 "Breast ER+/HER- ## Capecitabine" = "cn_simple_chr21_p_gt75",
 "Pan-Cancer ## Nivolumab" = "rna_geneset_gene_set_t_cell_gep_6_lt25", 
 "Lung NSCLC ## Anti-PD-1" = "neo_ct_lt75", 
 "Prostate ## Anti-AR" = "rna_geneset_HALLMARK_WNT_BETA_CATENIN_SIGNALING_gt50", 
 "Pan-Cancer ## Aromatase inhibitor" = "rna_geneset_HALLMARK_ANDROGEN_RESPONSE_lt50", 
 "Pan-Cancer ## Anti-EGFR" = "drivers_pathway_perplexity_total_gt75",
 "Colorectum ## Chemotherapy" = "hotspot_KRAS_position_25398284", 
 "Skin Melanoma ## Immunotherapy" = "drivers_pathway_IMMUNE_EVASION_gt0", 
 "Breast Triple Negative ## Chemotherapy" = "rna_geneset_KEGG_PENTOSE_PHOSPHATE_PATHWAY_gt50",
 "Lung NSCLC ## Chemotherapy" = "drivers_pathway_NRF2_gt0"    
)

- Interation Event to Highlight

In [ ]:
highlighters_combo <- 
c( "Pan-Cancer ## Alkaloid ## Platinum" = "cn_simple_chr11_q_gt75_or_purity_msIndelsPerMb_gt75", 
   "Prostate ## Anti-AR" = "chord_BRCA1_gt0_or_rna_geneset_KEGG_COMPLEMENT_AND_COAGULATION_CASCADES_gt25", 
   "Colon ## Chemotherapy" = "hotspot_KRAS_position_25398284_and_purity_tmbPerMb_gt50", 
   "Pan-Cancer ## Multikinase inhibitor" = "cn_simple_chr17_p_lt75_and_rna_geneset_KEGG_PROPANOATE_METABOLISM_lt25", 
   "Prostate ## Anti-AR" = "rna_geneset_HALLMARK_OXIDATIVE_PHOSPHORYLATION_lt75_and_rna_geneset_HALLMARK_WNT_BETA_CATENIN_SIGNALING_gt50",
   "Lung NSCLC ## Anti-PD-1" = "cn_simple_chr3_p_lt75_and_neo_ct_lt75", 
   "Breast Triple Negative ## Chemotherapy" = "cn_simple_chr22_q_gt25_and_rna_geneset_KEGG_PENTOSE_PHOSPHATE_PATHWAY_gt50"
 )

In [ ]:
now <- c()
for( i in names(highlighters_univariate)){
    now <- c(now, paste0(i, "--", highlighters_univariate[i]))
}

In [ ]:
map <- c("e_nr" = "Event + No DCB", "e_r" = "Event + DCB", "ne_nr" = "No Event + No DCB", "ne_r" = "No Event + DCB")

In [ ]:
tmp %>% fi(!feature %in% names(feature_map))

In [ ]:
tmp <- 
results %>% 
 se(cohortGo, feature, e_nr, ne_nr, e_r, ne_r, p_bh, group) %>% 
 ga(event, ct, -cohortGo, -feature, -p_bh, -group) %>% 
 mu(event2 = factor(map[event], levels = unname(map)),  event = !m("No Event", event2), dcb = !m("No DCB", event2)) %>% 
 rw() %>% mu(then = paste0(cohortGo, "--", feature)) %>% ug() %>% 
 fi( then %in% now) %>%
 gb(cohortGo, feature) %>% 
 mu(nr = max(ct*(as.numeric(event2 == "Event + No DCB")))) %>% 
 ug() %>% 
 mu( nice_cohort = as.character(gsub("##", "/", cohortGo)), nice_feature = unlist(feature_map[feature])) %>% 
 ug()

In [ ]:
head(tmp)

In [ ]:
feature_map <- 
list("cn_simple_chr21_p_gt75" = "High Chr 21p CN        ",
  "rna_geneset_KEGG_PENTOSE_PHOSPHATE_PATHWAY_gt50" = "High RNA Pentose Phosphate",
  "hotspot_KRAS_position_25398284" = "KRAS G12D Mutation      ",
  "neo_ct_lt75" = "High RNA Neoantigen Load  ",
  "drivers_pathway_NRF2_gt0" = "High NRF2 Pathway Drivers",
  "drivers_pathway_perplexity_total_gt75" = "High Total Pathway Drivers",
  "rna_geneset_HALLMARK_ANDROGEN_RESPONSE_lt50" = "Low RNA Androgen Response",
  "rna_geneset_gene_set_t_cell_gep_6_lt25" = "Low RNA T-cell Infiltration",
  "rna_geneset_KEGG_PHENYLALANINE_METABOLISM_lt25" = "Low RNA Phyenylalanine Metabolism",
  "rna_geneset_KEGG_TGF_BETA_SIGNALING_PATHWAY_gt25" = "High RNA TGFB Signalling",
  "rna_geneset_HALLMARK_WNT_BETA_CATENIN_SIGNALING_gt50" = "High RNA WNT Beta Catenin Signal", 
  "drivers_pathway_IMMUNE_EVASION_gt0" = "High Drivers Immune Evasion")

In [ ]:
cohort_order <- c("Breast ER+/HER- / Capecitabine", "Prostate / Abiraterone", "Pan-Cancer / Pazopanib", "Pan-Cancer / Nivolumab",
  "Lung NSCLC / Anti-PD-1", "Prostate / Anti-AR", "Pan-Cancer / Aromatase inhibitor", "Pan-Cancer / Anti-EGFR", 
  "Colorectum / Chemotherapy", "Skin Melanoma / Immunotherapy", "Breast Triple Negative / Chemotherapy", "Lung NSCLC / Chemotherapy")
  
feature_order <- c("High Chr 21p CN        ", "High RNA TGFB Signalling", "Low RNA Phyenylalanine Metabolism", "Low RNA T-cell Infiltration",
  "High RNA Neoantigen Load  ", "High RNA WNT Beta Catenin Signal", "Low RNA Androgen Response", "High Total Pathway Drivers", 
  "KRAS G12D Mutation      ", "High Drivers Immune Evasion", "High RNA Pentose Phosphate", "High NRF2 Pathway Drivers")

In [ ]:
tmp <- 
results %>% 
 se(cohortGo, feature, e_nr, ne_nr, e_r, ne_r, p_bh, group) %>% 
 ga(event, ct, -cohortGo, -feature, -p_bh, -group) %>% 
 mu(event2 = factor(map[event], levels = unname(map)),  event = !m("No Event", event2), dcb = !m("No DCB", event2)) %>% 
 rw() %>% mu(then = paste0(cohortGo, "--", feature)) %>% ug() %>% 
 fi( then %in% now) %>%
 gb(cohortGo, feature) %>% 
 mu(nr = max(ct*(as.numeric(event2 == "Event + No DCB")))) %>% 
 ug() %>% 
 mu( nice_cohort = as.character(gsub("##", "/", cohortGo)), nice_feature = unlist(feature_map[feature]), 
     cohort_feature = paste0(nice_cohort, " ## ", nice_feature)) %>% 
 mu( nice_cohort = factor(nice_cohort, levels = rev(cohort_order)), 
     nice_feature = factor(nice_feature, levels = rev(feature_order))) %>% 
 ug()

In [ ]:
alphas <- c("FALSE" = .6, "TRUE" = .8)
colors <- c("FALSE" = "black", "TRUE" = "black")
response <- c("FALSE" = "#e52f28", "TRUE" = "#7AABD3")

In [ ]:
a <- 
tmp %>% fi(grepl("No Event", event2)) %>% 
 mu(dcb = factor(dcb, levels = c(TRUE, FALSE))) %>% 
 mu(cohort_feature = paste0(cohortGo, "\n", feature, "\n ")) %>% 
 ggplot( aes( x = nice_cohort, y = ct, alpha = event, fill = dcb, color = event)) +
 geom_bar(stat = "identity", width = 0.7) + 
 coord_flip() + 
 scale_fill_manual( values = response) +  
 scale_alpha_manual( values = alphas) + 
 scale_color_manual( values = colors) + 
 scale_y_reverse() + theme_minimal() + 
 theme_minimal() +
 theme(
    plot.title = element_text(hjust = 0.5),       # center the title
    panel.grid.major = element_blank(),           # remove major grid lines
    panel.grid.minor = element_blank(),
    axis.title.y = element_blank(),     # Remove x-axis title
    legend.position = "none"
  ) + 
 labs(y = "Number of Patients", title = "No Event") + 
   geom_text(aes(label = ct), 
            position = position_stack(vjust = 0.5),  # center label inside bar
            color = "black")

In [ ]:
b <- 
tmp %>% fi(!grepl("No Event", event2)) %>% 
 mu(dcb = factor(dcb, levels = c(TRUE, FALSE))) %>% 
 ggplot( aes( y = nice_feature, x = ct, alpha = event, fill = dcb, color = event)) + 
 geom_bar(stat = "identity", width = 0.7) + 
 scale_fill_manual( values = response) +  
 scale_alpha_manual( values = alphas) + 
 scale_color_manual( values = colors)  +   
 theme_minimal() +
 theme(
    plot.title = element_text(hjust = 0.5),       # center the title
    panel.grid.major = element_blank(),           # remove major grid lines
    panel.grid.minor = element_blank(),
    axis.title.y = element_blank(),     # Remove x-axis title
    legend.position = "none"
  ) + 
 labs(x = "Number of Patients", title = "Event") + 
 geom_text(aes(label = ct), position = position_stack(vjust = 0.5),  color = "black")

In [ ]:
c <- (a | b) + plot_layout(heights = c(4, 4)) 

In [ ]:
options(repr.plot.width = 9, repr.plot.height = 5)
c
ggsave( "try_it.png", plot = c, width = 9, height = 5)

In [ ]:
getwd()

In [ ]:
library(patchwork)

In [ ]:
alphas <- c("FALSE" = .4, "TRUE" = 1)
colors <- c("FALSE" = "grey", "TRUE" = "black")
response <- c("FALSE" = "#e52f28", "TRUE" = "#7AABD3")

In [ ]:
options(repr.plot.width =13, repr.plot.height = 4)

In [ ]:
plotter <- function( df, label = "Treatment Examples\n", theme = go_theme, ylab_pt = 30 ){
 
 ggplot(data = df, aes(x = event2, y = ct, alpha = event, fill = dcb, color = event)) + 
  facet_wrap(~fct_reorder(cohortGo, p_bh, .fun = mean) + str_wrap(gsub("_", " ", gsub("rna_geneset_", "",feature)), width = 45) , ncol = 4) +    
  geom_bar(stat = "identity") + 
  scale_fill_manual( values = response) +  
  scale_alpha_manual( values = alphas) + 
  scale_color_manual( values = colors) + 
  geom_text(aes(label = ct), vjust = -0.5, color = "black", size = 3) + 
  scale_y_continuous(
    name = "Number of Patients\n",
    sec.axis = sec_axis(~ ., name = label), 
    expand = expansion(mult = c(0.1, 0.2))
  ) + 
  geom_text(aes(x = 1.3, y = ylab_pt, label = paste0("Adjusted p-value:\n", formatC(p_bh, format = "e", digits = 1))), vjust = -0.5, color = "black", size = 3) + 
  labs(title = "Highlight Individual Molecular Events with Rare Responders", x = "Event Group") +
  theme
}

In [ ]:
top_theme <-
go_theme + 
 theme(strip.background = element_rect(fill = "white", color = "black"), 
       legend.position = "none", 
       axis.title.x = element_blank(),
       axis.text.x = element_blank(),
       axis.ticks.x = element_blank())

In [ ]:
mid_theme <-top_theme + theme(plot.title = element_blank())

In [ ]:
bottom_theme <-
go_theme + 
 theme(strip.background = element_rect(fill = "white", color = "black"), 
       legend.position = "none", 
       plot.title = element_blank())

In [ ]:
top <- plotter(tmp %>% fi(group == "treatment"), theme = top_theme, label = "Treatment\n",  ylab_pt = 45)
mid <- plotter(tmp %>% fi(group == "mechanism"), theme = mid_theme, label = "Mechanism\n", ylab_pt = 28)
bottom <- plotter(tmp %>% fi(group == "type"), theme = bottom_theme, label = "Type\n", ylab_pt = 85)

In [ ]:
options(repr.plot.width =14, repr.plot.height = 9)
library(patchwork)

In [ ]:
univariate <- (top / mid / bottom ) + plot_layout(heights = c(4, 4, 5)) 

In [ ]:
getwd()

In [ ]:
univariate
ggsave( "univariate_examples.png", plot = univariate, width = 14, height = 9)

# Generally Bad Prognostic Features 